In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, ReLU, MaxPooling2D, Flatten, Dense
from tensorflow.keras import Model

In [ ]:
class CNN(tf.keras.Model):
    def __init__(self, input_channels, num_classes):
        super(CNN, self).__init__()

        # Convolutional layers
        self.conv1 = Conv2D(
            32, kernel_size=3, strides=1, padding="same", activation="relu"
        )
        self.pool1 = MaxPooling2D(pool_size=2, strides=2)

        self.conv2 = Conv2D(
            64, kernel_size=3, strides=1, padding="same", activation="relu"
        )
        self.pool2 = MaxPooling2D(pool_size=2, strides=2)

        # Fully connected layers
        self.flatten = Flatten()
        self.fc1 = Dense(512, activation="relu")
        self.fc2 = Dense(num_classes)

    def call(self, inputs):
        # Convolutional layers
        x = self.pool1(self.conv1(inputs))
        x = self.pool2(self.conv2(x))

        # Flatten for fully connected layers
        x = self.flatten(x)

        # Fully connected layers
        x = self.fc1(x)
        x = self.fc2(x)

        return x

    def train_model(
        self, train_data, train_labels, num_epochs=100, learning_rate=0.001
    ):
        # Define loss function and optimizer
        criterion = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
        optimizer = tf.keras.optimizers.Adam(learning_rate)

        for epoch in range(num_epochs):
            # Convert data to TensorFlow tensors
            inputs = tf.constant(train_data, dtype=tf.float32)
            labels = tf.constant(train_labels, dtype=tf.int64)

            # Forward pass
            with tf.GradientTape() as tape:
                outputs = self(inputs)
                loss = criterion(labels, outputs)

            # Backward and optimize
            gradients = tape.gradient(loss, self.trainable_variables)
            optimizer.apply_gradients(zip(gradients, self.trainable_variables))

            if (epoch + 1) % 1 == 0:
                print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.numpy():.4f}")

    def predict(self, input_data):
        # Make predictions for input_data
        predictions = self(input_data)
        return tf.argmax(predictions, axis=1).numpy()

In [ ]:
# Generate synthetic training data for TensorFlow
num_samples = 100
image_height = 32
image_width = 32
input_channels = 3
num_classes = 10

train_data = tf.random.normal((num_samples, image_height, image_width, input_channels))
train_labels = tf.random.uniform(
    (num_samples,), minval=0, maxval=num_classes, dtype=tf.int64
)

# Displaying the shape of the generated data (optional)
print("Train Data Shape:", train_data.shape)
print("Train Labels Shape:", train_labels.shape)

# Example usage for TensorFlow:
model_cnn = CNN(input_channels=input_channels, num_classes=num_classes)
model_cnn.train_model(train_data, train_labels)

# Example test data
test_data_cnn = tf.random.normal((1, image_height, image_width, input_channels))

# Make predictions with TensorFlow model
predictions_cnn = model_cnn.predict(test_data_cnn)
print(
    "Predictions (CNN):", predictions_cnn
)  # In TensorFlow, represents the class index directly